In [100]:
import pandas as pd

In [101]:
df = pd.read_csv('../fx_history.csv')
#df = df.drop(columns = ['Unnamed: 0'])
df.head(2)

,date,type,volume,position_id,price,commission,swap,profit,fee,symbol
0,2023-10-06,1,0.05,629162023,182.292,-0.14,0.0,0.54,0.0,GBPJPY
1,2023-10-06,0,0.10,629060972,1822.240,-0.28,0.0,0.00,0.0,XAUUSD


In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_growth_over_time(dataframe: pd.DataFrame,
                          x_column: str,
                          y_column: str,
                          title: str = "Profit Over Time",
                          yaxis_title: str = "Growth") -> go.Figure:
    """
    Create a bar plot to visualize growth over time.

    Args:
        dataframe (pd.DataFrame): A DataFrame containing the data to be plotted.
        x_column (str): The name of the column to be used as the x-axis.
        y_column (str): The name of the column to be used as the y-axis.
        title (str, optional): The title of the plot. Default is "Profit Over Time".
        yaxis_title (str, optional): The title of the y-axis. Default is "Growth".

    Returns:
        go.Figure: A Plotly figure representing the bar plot.

    Example:
        df = pd.DataFrame({
            'Date': ['2023-09-01', '2023-09-02', '2023-09-03'],
            'Profit': [100, 150, 120]
        })

        fig = plot_growth_over_time(df, x_column='Date', y_column='Profit', title='Profit Over Time', yaxis_title='Profit')
        fig.show()

    Output:
        (Bar plot visualization of growth over time)
    """
    # Create a new Plotly figure
    fig = go.Figure()

    # Create a bar plot
    fig.add_trace(go.Bar(x=dataframe[x_column], y=dataframe[y_column], name='Growth'))

    # Customize the layout
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title=yaxis_title,
        showlegend=True
    )

    return fig


In [102]:
df.loc[:,'date'] = pd.to_datetime(df['date'])

In [97]:
def monthly_percentage_growth(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate monthly percentage growth and related metrics from a given DataFrame.

    Parameters:
    - df (pandas.DataFrame): Input DataFrame containing at least the columns ['date', 'type', 'profit', 'swap', 'commission', 'fee'].

    Returns:
    - pandas.DataFrame: A DataFrame with monthly percentage growth and related metrics.
    """
    
    # Convert 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    # Calculate total deposits based on type==2 and store in variable
    deposit = sum(df.loc[df.type == 2].profit.tolist())
    
    # Filter out rows where type is 2
    df = df.loc[df['type'] != 2].copy()  # using .copy() to avoid SettingWithCopyWarning
    
    # Extract month and year from the 'date' column and create respective columns
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    
    # Calculate the total profit by adding the profit, swap, commission, and fee columns
    df.loc[:, 'total_profit'] = df['profit'] + df['swap'] + df['commission'] + df['fee']
    
    # Group by month and year, then sum the total profit for each group
    df_month_percentage_growth = df.groupby(by=['month', 'year'])['total_profit'].sum().reset_index()
    
    # Calculate the cumulative monthly growth
    df_month_percentage_growth.loc[:, 'cummulative_monthly_growth'] = df_month_percentage_growth['total_profit'].cumsum()
    
    # Calculate the daily balance by adding deposit to cumulative monthly growth
    df_month_percentage_growth.loc[:, 'daily_balance'] =  deposit + df_month_percentage_growth['cummulative_monthly_growth']
    
    # Calculate the percentage monthly growth based on deposit
    df_month_percentage_growth.loc[:, 'percentage_monthly_growth_%'] = round((df_month_percentage_growth['cummulative_monthly_growth'] / deposit), 3)
    
    # Calculate the cumulative monthly percentage growth based on the percentage monthly growth
    df_month_percentage_growth.loc[:, 'cummulative_monthly_percentage_growth_%'] = round(df_month_percentage_growth['percentage_monthly_growth_%'].diff() * 100, 3)
    
    # Sort the DataFrame by year and month in descending order
    df_month_percentage_growth = df_month_percentage_growth.sort_values(by=['year', 'month'], ascending=False)
    
    # Slice the DataFrame to get the first two rows
    df_month_cum_growth = df_month_percentage_growth.iloc[:2]

    return df_month_cum_growth

In [98]:
df_month_cum_growth = monthly_percentage_growth(df)
df_month_cum_growth

,month,year,total_profit,cummulative_monthly_growth,daily_balance,percentage_monthly_growth_%,cummulative_monthly_percentage_growth_%
1,10,2023,119.15,-549.74,1240.26,-0.307,6.7
0,9,2023,-668.89,-668.89,1121.11,-0.374,NaN


In [99]:
current_month_growth = df_month_cum_growth['cummulative_monthly_percentage_growth_%'].to_list()[0]
previous_month_growth = df_month_cum_growth['cummulative_monthly_percentage_growth_%'].to_list()[1]
st.metric(label = 'monthly_growth', value = f"{current_month_growth} %", delta = f"{previous_month_growth} %" )

In [94]:
df['date'] = pd.to_datetime(df['date'])
deposit  =sum(df.loc[df.type == 2].profit.tolist())
df = df.loc[df['type'] != 2]
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df.loc[:, 'total_profit'] = df['profit'] + df['swap'] + df['commission'] + df['fee']
df_month_percentage_growth = df.groupby(by = ['month','year'])['total_profit'].sum().reset_index()
df_month_percentage_growth.loc[:, 'cummulative_monthly_growth'] = df_month_percentage_growth['total_profit'].cumsum()
df_month_percentage_growth.loc[:, 'daily_balance'] =  deposit + df_month_percentage_growth['cummulative_monthly_growth']
df_month_percentage_growth.loc[:,'percentage_monthly_growth_%'] = round((df_month_percentage_growth['cummulative_monthly_growth'] / deposit),3)
df_month_percentage_growth.loc[:,'cummulative_monthly_percentage_growth_%'] = round(df_month_percentage_growth['percentage_monthly_growth_%'].diff() * 100, 3)
df_month_percentage_growth = df_month_percentage_growth.sort_values(by = ['year','month'], ascending = False)
df_month_cum_growth = df_month_percentage_growth.iloc[:2]



In [95]:
df_month_cum_growth

,month,year,total_profit,cummulative_monthly_growth,daily_balance,percentage_monthly_growth_%,cummulative_monthly_percentage_growth_%
1,10,2023,119.15,-549.74,1240.26,-0.307,6.7
0,9,2023,-668.89,-668.89,1121.11,-0.374,NaN


In [34]:
print(deposit)


0


In [17]:
df.shape

(961, 14)

In [4]:
import plotly.graph_objects as go
df['date'] = pd.to_datetime(df['date'])
result_df = df.loc[df['type'] != 2]
# Extract week and month
result_df.loc[:, 'total_profit'] = result_df['profit'] + result_df['swap'] + result_df['commission'] + result_df['fee']
result_df['week'] = result_df['date'].dt.isocalendar().week
result_df['month'] = result_df['date'].dt.month
weekly_growth = result_df.groupby(by ='month')['total_profit'].sum().reset_index()
weekly_growth['color'] = weekly_growth['total_profit'].apply(lambda x: 'green' if x >= 0 else 'red')
# Create the bar chart
fig = go.Figure(data=[go.Bar(
    x=weekly_growth['month'],
    y=weekly_growth['total_profit'],
    marker_color=weekly_growth['color'],  # Set the bar color based on the 'color' column
)])

# Customize the chart appearance
fig.update_layout(
    title="Weekly Growth",
    xaxis_title="month",
    yaxis_title="Profit",
)

# Show the chart
fig.show()


C:\Users\Asus\AppData\Local\Temp\ipykernel_43376\750893525.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.loc[:, 'total_profit'] = result_df['profit'] + result_df['swap'] + result_df['commission'] + result_df['fee']
C:\Users\Asus\AppData\Local\Temp\ipykernel_43376\750893525.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['week'] = result_df['date'].dt.isocalendar().week
C:\Users\Asus\AppData\Local\Temp\ipykernel_43376\750893525.py:7: SettingWithCopyWarning: 
A value is trying to

In [5]:
def create_weekly_growth_chart(df: pd.DataFrame, weeklygrowth = True) -> Figure:
    """
    Create and display a weekly growth bar chart from a DataFrame.

    Args:
        df (DataFrame): The DataFrame containing the data with 'date', 'type', and 'profit' columns.

    Returns:
        Figure: A Plotly Figure object representing the weekly growth chart.
    """
    # Convert the 'date' column to DateTime
    df['date'] = pd.to_datetime(df['date'])

    # Filter the DataFrame by 'type' column
    result_df = df.loc[df['type'] != 2]

    # Calculating total profit 
    result_df.loc[:, 'total_profit'] = result_df['profit'] + result_df['swap'] + result_df['commission'] + result_df['fee']
    # Extract week and month
    result_df['week'] = result_df['date'].dt.isocalendar().week
    result_df['month'] = result_df['date'].dt.month

    if weeklygrowth == True:
        # Calculate weekly growth
        weekly_growth = result_df.groupby(by='week')['total_profit'].sum().reset_index()
        weekly_growth['color'] = weekly_growth['total_profit'].apply(lambda x: 'green' if x >= 0 else 'red')

        # Create the bar chart
        fig = go.Figure(data=[go.Bar(
            x=weekly_growth['week'],
            y=weekly_growth['total_profit'],
            marker_color=weekly_growth['color'],  # Set the bar color based on the 'color' column
        )])
            # Customize the chart appearance
        fig.update_layout(
            title="Weekly Growth",
            xaxis_title="Week",
            yaxis_title="Profit",
        )

    else:
        # Calculate monthly growth
        monthly_growth = result_df.groupby(by='month')['total_profit'].sum().reset_index()
        monthly_growth['color'] = monthly_growth['total_profit'].apply(lambda x: 'green' if x >= 0 else 'red')

        # Create the bar chart
        fig = go.Figure(data=[go.Bar(
            x=monthly_growth['month'],
            y=monthly_growth['total_profit'],
            marker_color=monthly_growth['color'],  # Set the bar color based on the 'color' column
        )])
        # Customize the chart appearance
        fig.update_layout(
            title="Monthly Growth",
            xaxis_title="month",
            yaxis_title="Profit",
        )

    return fig

NameError: name 'Figure' is not defined

In [11]:
weekly_growth

,month,profit,color
0,9,-458.85,red


In [13]:
result_df

,date,type,volume,position_id,price,commission,swap,profit,fee,symbol,week,month
0,2023-09-29,0,0.05,628063529,1855.01,-0.14,0.0,-2.13,0.0,XAUUSD,39,9
1,2023-09-29,0,0.02,628003987,1868.62,-0.06,0.0,3.27,0.0,XAUUSD,39,9
2,2023-09-29,0,0.10,627946859,1868.13,-0.28,0.0,0.00,0.0,XAUUSD,39,9
3,2023-09-29,1,0.05,627946859,1869.14,-0.14,0.0,4.77,0.0,XAUUSD,39,9
4,2023-09-29,1,0.02,627946859,1869.14,-0.06,0.0,1.91,0.0,XAUUSD,39,9
...,...,...,...,...,...,...,...,...,...,...,...,...
668,2023-09-01,0,0.05,624029498,1949.68,-0.14,0.0,10.95,0.0,XAUUSD,35,9
669,2023-09-01,1,0.05,624029265,1951.60,-0.14,0.0,0.00,0.0,XAUUSD,35,9
670,2023-09-01,0,0.02,624029265,1951.55,-0.06,0.0,0.09,0.0,XAUUSD,35,9
671,2023-09-01,1,0.05,624078663,1934.88,-0.14,0.0,0.00,0.0,XAUUSD,35,9


In [ ]:
deposit  =sum(df.loc[df.type == 2].profit.tolist())
deposit

In [ ]:
def total_trades(df):
    df_sys = df.loc[df['type'] == 0]
    total_trades = df_sys['position_id'].nunique()
    return total_trades


In [ ]:
df_sys = df.loc[df['type'] == 0]

In [ ]:
total_trades = df_sys['position_id'].nunique()
total_trades

In [15]:
def total_trade_counts(date: pd.DataFrame) -> int:
    """
    Calculate the total number of trades made on a given date.

    Args:
        df (pd.DataFrame): A DataFrame containing trading data with a 'type', 'date', and 'position_id' column.

    Returns:
        int: The total number of trades made on the specified date.
    """
    # Filter rows with 'type' equal to 0 ( 0 represents trade)
    df_trade = df.loc[df.type == 0]

    # Group by 'date' and 'position_id', and count the number of trades ('counts' column)
    df_trade = df_trade.groupby(by = ['date','position_id']).size().reset_index(name='counts')

    # Sort the DataFrame by trade counts in descending order
    df_trade = df_trade.sort_values('counts', ascending = False)

    # Get the total number of trades by counting the number of rows in the DataFrame
    total_trades  = df_trade.shape[0]

    return total_trades 

In [16]:
total_trade_counts(df)

299

In [17]:
def total_trades(df: pd.DataFrame) -> int:
    """
    Calculate the total number of unique trades in a DataFrame.

    Args:
        df (pd.DataFrame): A DataFrame containing trading data with a 'type' column.

    Returns:
        int: The total number of unique trades.

    Example:
        df = pd.DataFrame({
            'position_id': [1, 2, 3, 4, 5],
            'type': [0, 1, 0, 0, 1]
        })

        total = total_trades(df)
        print(total)

    Output:
        3
    """
    # Filter rows where 'type' is equal to 0 (assuming 'type' 0 represents trades)
    df_sys = df.loc[df['type'] == 0]

    # Calculate the total number of unique trades based on 'position_id'
    total_trades = df_sys['position_id'].nunique()

    return total_trades

In [21]:
total_trades(df)

299

In [ ]:
def get_pip_profit_loss(df):

    df['total_profit'] = df.commission + df.swap + df.profit
    df_pip_cal = df[['date','position_id','price','symbol','total_profit']]
    df_pip_cal['profit_loss'] = df_pip_cal['total_profit'].apply(lambda x: 'neg' if x < 0 else 'pos')
    df_pip_cal['date_pos_id'] = df_pip_cal.date.astype(str) + '_' + df_pip_cal.position_id.astype(str) + '_'+ df_pip_cal.profit_loss
    df_pip_cal_gb = df_pip_cal.groupby('date_pos_id')['price'].agg(max_price=max, min_price=min).reset_index()
    df_pip_cal_gb['pips'] = (df_pip_cal_gb['max_price'] - df_pip_cal_gb['min_price'])*10
    # Split the 'date_position' column into two columns: 'date' and 'position_id'
    df_pip_cal_gb[['date', 'position_id', 'p/l']] = df_pip_cal_gb['date_pos_id'].str.split('_', expand=True)

    # Drop the 'date_position' column if you no longer need it
    df_pip_cal_gb.drop('date_pos_id', axis=1, inplace=True)
    df_pip_cal_gb = df_pip_cal_gb[['date','pips','p/l']]
    # Make 'pips' negative where 'p/l' is 'neg'
    df_pip_cal_gb.loc[df_pip_cal_gb['p/l'] == 'neg', 'pips'] = -df_pip_cal_gb.loc[df_pip_cal_gb['p/l'] == 'neg', 'pips']
    return df_pip_cal_gb

get_pip_profit_loss(df)


In [ ]:
df['total_profit'] = df.commission + df.swap + df.profit



In [ ]:
df_pip_cal = df[['date','position_id','price','symbol','total_profit']]
df_pip_cal.head(3)

In [ ]:
df_pip_cal['profit_loss'] = df_pip_cal['total_profit'].apply(lambda x: 'neg' if x < 0 else 'pos')


In [ ]:
df_pip_cal['date_pos_id'] = df_pip_cal.date.astype(str) + '_' + df_pip_cal.position_id.astype(str) + '_'+ df_pip_cal.profit_loss
df_pip_cal.head(3)

In [ ]:
df_pip_cal_gb = df_pip_cal.groupby('date_pos_id')['price'].agg(max_price=max, min_price=min).reset_index()
df_pip_cal_gb['pips'] = (df_pip_cal_gb['max_price'] - df_pip_cal_gb['min_price'])*10
df_pip_cal_gb

In [ ]:
# Split the 'date_position' column into two columns: 'date' and 'position_id'
df_pip_cal_gb[['date', 'position_id', 'p/l']] = df_pip_cal_gb['date_pos_id'].str.split('_', expand=True)

# Drop the 'date_position' column if you no longer need it
df_pip_cal_gb.drop('date_pos_id', axis=1, inplace=True)

In [ ]:
df_pip_cal_gb.head()

In [ ]:
df_pip_cal_gb = df_pip_cal_gb[['date','pips','p/l']]
df_pip_cal_gb.head()

In [ ]:
df_pip_cal_gb = df_pip_cal_gb.sort_values('date', ascending=False)
df_pip_cal_gb.head()

In [ ]:
# Make 'pips' negative where 'p/l' is 'neg'
df_pip_cal_gb.loc[df_pip_cal_gb['p/l'] == 'neg', 'pips'] = -df_pip_cal_gb.loc[df_pip_cal_gb['p/l'] == 'neg', 'pips']

In [ ]:
df_pip_cal_gb.head()

In [ ]:
df_swaps = df.groupby(by='date')['swap'].sum().reset_index()
df_swaps = df_swaps.sort_values(by=['date'], ascending = False)
swaps = df_swaps['swap'][0]
swaps

In [ ]:
df_growth = get_portfolio_growth(df,profit = False)
current_portfolio_value = df_growth.sort_values(by = 'date', ascending = False)['growth'].to_list()[0]
total_profit_or_Loss = round(current_portfolio_value/(deposit)*100,1)
total_profit_or_Loss_eur = round((current_portfolio_value - deposit),2)
total_profit_or_Loss_eur

In [ ]:
total_profit_or_Loss -100

In [ ]:
df_profit

In [ ]:
df_growth

In [ ]:
Percentage_Gain_or_Loss = round(profit_or_Loss/(current_portfolio_value - abs(profit_or_Loss) )*100,1)
Percentage_Gain_or_Loss

In [ ]:
df_commission = df.groupby(by='date')['commission'].sum().reset_index()
df_commission = df_commission.sort_values(by='date', ascending = False).reset_index()
df_commission = df_commission.drop(columns = ['index'])
commissions = round(df_commission.commission.to_list()[0],2)
commissions



In [ ]:
df_sym = df.loc[df['type'] == 0]
commodity = df_sym.groupby(by=['date','symbol']).size().reset_index(name = 'count')
result = commodity.groupby('date').agg({'symbol': list, 'count': list}).reset_index()
result

In [ ]:
result_ord = result.sort_values(by=['date'], ascending=False).reset_index()
result_ord = result_ord.drop(columns = ['index'])

result_ord

In [ ]:
symbol_list = result_ord.loc[1, 'symbol']
symbol_len = len(symbol_list)
if symbol_len != 1:
    for i in range(symbol_len):

        symbol_dict ={'symbol':result_ord.loc[1, 'symbol'][i],
                    'count':result_ord.loc[1, 'count'][i]}

    symbol_dict
else:
    symbol_list = result_ord.loc[1, 'symbol']
        

symbol_dict

In [ ]:
# Assuming 'result_ord' is your DataFrame and you want to process the second row
symbol_dict = {}  # Create an empty dictionary to store symbol-count pairs

for i in range(len(result_ord.loc[1, 'symbol'])):
    symbol = result_ord.loc[1, 'symbol'][i]
    count = result_ord.loc[1, 'count'][i]
    
    # Update the symbol_dict with the symbol-count pair
    symbol_dict[symbol] = count

# Print the resulting dictionary
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(symbol_dict, orient='index', columns=['count']).reset_index()
df.rename(columns={'index': 'symbol'}, inplace=True)

# Print the DataFrame
print(df)

In [ ]:
import pandas as pd

def create_symbol_count_dataframe(dataframe, row_index):
    """
    Converts symbol and count data from a specific row of a DataFrame
    into a pandas DataFrame.

    Args:
        dataframe (pd.DataFrame): The DataFrame containing the data.
        row_index (int): The index of the row to process.

    Returns:
        pd.DataFrame: A DataFrame containing symbol-count pairs.
    """
    symbol_dict = {}  # Create an empty dictionary to store symbol-count pairs

    for i in range(len(dataframe.loc[row_index, 'symbol'])):
        symbol = dataframe.loc[row_index, 'symbol'][i]
        count = dataframe.loc[row_index, 'count'][i]

        # Update the symbol_dict with the symbol-count pair
        symbol_dict[symbol] = count

    # Convert the dictionary into a DataFrame
    df = pd.DataFrame.from_dict(symbol_dict, orient='index', columns=['count']).reset_index()
    df.rename(columns={'index': 'symbol'}, inplace=True)

    return df


row_index = 1  # Choose the row you want to process
symbol_count_df = create_symbol_count_dataframe(result_ord, row_index)

# Print the resulting DataFrame
print(symbol_count_df)


In [ ]:
len(symbol_count_df)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
#result_df = df.loc[df['type'] != 2]
# result_df.loc[:, 'total_profit'] = result_df['profit'] + result_df['swap'] + result_df['commission'] + result_df['fee']
# result_df = result_df.groupby('date')['total_profit'].sum().reset_index()


deposit  =sum(df.loc[df.type == 2].profit.tolist())
#result_df.loc[:, 'total_profit'] = result_df['total_profit'] + deposit


# result_df.columns = ['date', 'growth']
# result_df = result_df.sort_values(by='date', ascending = False )

# Print the result

deposit

In [ ]:
initial_deposit = 700
result_df['daily_balance'] = initial_deposit + result_df['total_profit'].cumsum()

In [ ]:
result_df

In [ ]:
df['date'] = pd.to_datetime(df['date'])
result_df = df.loc[df['type'] != 2]
result_df.loc[:, 'total_profit'] = result_df['profit'] + result_df['swap'] + result_df['commission'] + result_df['fee']
result_df = result_df.groupby('date')['total_profit'].sum().reset_index()


deposit  =df.loc[df.type == 2].profit.tolist()[0]
result_df.loc[:, 'daily_balance'] =  deposit + result_df['total_profit'].cumsum()
result_df.columns = ['date', 'daily_profit','growth']


#result_df.columns = ['date', 'growth']
result_df = result_df.sort_values(by='date', ascending = False )

# Print the result
result_df

In [ ]:
df_commision = df.groupby(by='date')['commission'].sum().reset_index()
commisions = df_commision.commission.sum()
commisions

In [ ]:
df_profitloss  =df.copy()

In [ ]:
def profit_loss_df(df):

    df_profitloss  =df.copy()
    df_profitloss.loc[:, 'total_profit'] = df['profit'] + df['swap'] + df['commission'] + df['fee']
    df_profitloss = df_profitloss.loc[df_profitloss.type ==0]
    df_profitloss = df_profitloss.groupby(by='date')['total_profit'].sum().reset_index()

    # Create new DataFrames for positive and negative profits
    positive_df = df_profitloss[df_profitloss['total_profit'] > 0].copy()
    negative_df = df_profitloss[df_profitloss['total_profit'] < 0].copy()

    # Rename the 'Profit' column in each new DataFrame
    positive_df.rename(columns={'Profit': 'Profit'}, inplace=True)
    negative_df.rename(columns={'Profit': 'Loss'}, inplace=True)

    # Reset the index for both new DataFrames
    positive_df.reset_index(drop=True, inplace=True)
    negative_df.reset_index(drop=True, inplace=True)

    # positive_df = positive_df[['date','profit']]
    # negative_df = negative_df[['date','profit']]

    return positive_df, negative_df

positive_df, negative_df = profit_loss_df(df)


In [ ]:
negative_df

In [ ]:
# number of trades taken 
df_trade = df.loc[df['type'] == 0]
df_trade = df_trade.groupby(by = 'date').size().reset_index(name='count')
df_trade = df_trade.sort_values(by='date', ascending = False)['count'].to_list()[0]
df_trade

In [ ]:

df_sym.head()

In [ ]:
def plot_piechat(df):

    df_sym = df.loc[df['type'] != 2]
    commodity = df_sym.groupby(by=['symbol']).size().reset_index(name  ='count')
    fig = px.pie(commodity, values= commodity['count'], names=commodity['symbol'], color_discrete_sequence=px.colors.sequential.RdBu)
    return fig

In [ ]:
df_profit = get_portfolio_growth(df)
df_profit.head()

In [ ]:
df_growth = get_portfolio_growth(df, profit = False)
df_growth.head()

In [ ]:
current_portfolio_value = df_growth.sort_values(by = 'date', ascending = False)['growth'].to_list()[0]
current_portfolio_value

In [ ]:
profil_or_Loss = df_profit.sort_values(by = 'date', ascending = False)['growth'].to_list()[0]
profil_or_Loss

In [ ]:
Percentage_Gain_or_Loss = round(profil_or_Loss/(current_portfolio_value + abs(profil_or_Loss))*100,1)
Percentage_Gain_or_Loss

In [ ]:
deposit  =df.loc[df.type == 2].profit.tolist()[0]
deposit

In [ ]:
result_df = df.loc[df.type != 2]

In [ ]:

result_df.loc[:, 'total_profit'] = result_df['profit'] + result_df['swap'] + result_df['commission'] + result_df['fee']


In [ ]:
result_df.head()

In [ ]:
# Using .loc to assign the values
result_df.loc[:, 'total_profit'] = result_df['total_profit'] + deposit


In [ ]:
result_df.head()

In [ ]:
def get_pip_profit_loss(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate pip profit or loss for each position in a DataFrame.
    This analysis would be only valid for it to make senc only if you trade with a
    constant lot size!

    Args:
        df (pd.DataFrame): A DataFrame containing the following columns:
            - 'date': Date of the position.
            - 'position_id': Unique identifier for each position.
            - 'price': Price of the position.
            - 'symbol': Symbol of the position.
            - 'commission': Commission for the position.
            - 'swap': Swap for the position.
            - 'profit': Profit for the position.

    Returns:
        pd.DataFrame: A DataFrame with the following columns:
            - 'date': Date of the position.
            - 'pips': Pip profit or loss for the position.
            - 'p/l': 'pos' for positive profit, 'neg' for negative profit.

    Example:
        df = pd.DataFrame({
            'date': ['2023-09-01', '2023-09-01', '2023-09-02'],
            'position_id': [1, 2, 3],
            'price': [100.0, 105.0, 98.0],
            'symbol': ['EURUSD', 'USDJPY', 'GBPUSD'],
            'commission': [5.0, 7.0, 4.0],
            'swap': [2.0, -1.0, 0.0],
            'profit': [10.0, -5.0, 8.0]
        })

        result_df = get_pip_profit_loss(df)
        print(result_df)

    Output:
            date  pips  p/l
        0  2023-09-01  50.0  pos
        1  2023-09-01 -50.0  neg
        2  2023-09-02   0.0  pos
    """
    # Calculate total profit for each position
    df['total_profit'] = df['commission'] + df['swap'] + df['profit']

    # Create a DataFrame with relevant columns
    df_pip_cal = df[['date', 'position_id', 'price', 'symbol', 'total_profit']]

    # Determine profit/loss status ('pos' or 'neg')
    df_pip_cal['profit_loss'] = df_pip_cal['total_profit'].apply(lambda x: 'neg' if x < 0 else 'pos')

    # Create a unique identifier for each position
    df_pip_cal['date_pos_id'] = df_pip_cal['date'].astype(str) + '_' + df_pip_cal['position_id'].astype(str) + '_' + df_pip_cal['profit_loss']

    # Group by the unique identifier and calculate max and min prices
    df_pip_cal_gb = df_pip_cal.groupby('date_pos_id')['price'].agg(max_price=max, min_price=min).reset_index()

    # Calculate pips based on max and min prices
    df_pip_cal_gb['pips'] = (df_pip_cal_gb['max_price'] - df_pip_cal_gb['min_price']) * 10

    # Split the 'date_position' column into two columns: 'date' and 'position_id'
    df_pip_cal_gb[['date', 'position_id', 'p/l']] = df_pip_cal_gb['date_pos_id'].str.split('_', expand=True)

    # Drop the 'date_position' column if you no longer need it
    df_pip_cal_gb.drop('date_pos_id', axis=1, inplace=True)

    # Select and reorder columns
    df_pip_cal_gb = df_pip_cal_gb[['date', 'pips', 'p/l']]

    # Make 'pips' negative where 'p/l' is 'neg'
    df_pip_cal_gb.loc[df_pip_cal_gb['p/l'] == 'neg', 'pips'] = -df_pip_cal_gb.loc[df_pip_cal_gb['p/l'] == 'neg', 'pips']

    return df_pip_cal_gb

In [ ]:
def calculating_pip_growth(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate pip growth based on input data.

    Args:
        data (pd.DataFrame): A DataFrame containing trading data with columns 'type', 'commission', 'swap',
                                'profit', 'date', 'position_id', 'symbol', and 'volume'.

    Returns:
        pd.DataFrame: A DataFrame containing calculated pip growth grouped by date and symbol.
    """
    # Filter out rows with 'type' equal to 2
    df_pip = data.loc[data.type != 2]

    # Calculate 'total_profit' as the sum of 'commission', 'swap', and 'profit'
    df_pip.loc[:, 'total_profit'] = df['commission'] + df['swap'] + df['profit']

    # Group by 'date', 'position_id', 'symbol', and 'volume', and sum 'total_profit'
    df_pip_gb = df_pip.groupby(by=['date','position_id','symbol','volume'])['total_profit'].sum().reset_index()

    # Group by 'date' and 'symbol', and sum 'volume' and 'total_profit'
    df_pip_daily_gb = df_pip_gb.groupby(['date', 'symbol'])[['volume','total_profit']].sum().reset_index()
    
    return df_pip_daily_gb


import pandas as pd
from pandas.core.series import Series
from pandas.core.frame import DataFrame

# Define a function to calculate pip_growth based on the symbol
def calculate_pip_growth(row: Series) -> float:
    """
    Calculates pip_growth based on the symbol.

    Args:
        row (pandas.Series): A row from the DataFrame containing 'symbol', 'volume', and 'total_profit'.

    Returns:
        float: The calculated pip_growth value.
    """
    if row['symbol'] == 'XAUUSD':
        return round(row['total_profit'] / (row['volume'] * 100 * 0.095), 1)
    elif row['symbol'] == 'GBPJPY':
        return round(row['total_profit'] / (row['volume'] * 100 * 0.063), 1)
    else:
        return None


def extract_latest_pip_growth(data: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the rows with the latest date from a DataFrame containing pip growth data.

    Args:
        data (pd.DataFrame): A DataFrame containing pip growth data with a 'date' column.

    Returns:
        pd.DataFrame: A DataFrame containing only the rows with the latest date.
    """
    # Sort the DataFrame by date in descending order (latest date first)
    data = data.sort_values('date', ascending=False)

    # Extract the rows with the latest date
    df_pip_latest = data.loc[data.date == data['date'].max()]

    return df_pip_latest

import plotly.graph_objects as go
import pandas as pd
import plotly.express as px


def plot_pip_daily_growth(data:pd.DataFrame):
    """
    Create a bar plot of pip growth with positive pip growth in green and negative pip growth in red.

    Args:
        data (pd.DataFrame): A DataFrame containing 'symbol' and 'pip_growth' columns.

    Returns:
        None: Displays the Plotly bar plot.
    """
    # extracting latest pip growth from data
    data = extract_latest_pip_growth(df_pip_daily_gb)
    # create a barplot
    
    # initilize the fig object
    fig = go.Figure()

    fig = go.Figure(data=[go.Bar(
                x=data.symbol, y=data.pip_growth,
                text=data.symbol,
                textposition='auto',
                marker=dict(
                color=data['pip_growth'].apply(lambda x: 'limegreen' if x >= 0 else 'lightcoral'),
                line=dict(color='black', width=1)))])# Border color for bars
    # Customize the layout
    fig.update_layout(
        plot_bgcolor='black',  # Background color
        paper_bgcolor='black',  # Plot area background color
        font=dict(color='white'),  # Text color
        xaxis_title='symbol',
        yaxis_title='pip_gains',
    )
    
    fig.show()





df_pip_daily_gb= calculating_pip_growth(df)
# Apply the function to create the pip_growth column
df_pip_daily_gb['pip_growth'] = df_pip_daily_gb.apply(calculate_pip_growth, axis=1)

plot_pip_daily_growth(df_pip_daily_gb)

In [ ]:
def get_portfolio_growth(df, profit = True) -> pd.DataFrame:
    
    
    df['date'] = pd.to_datetime(df['date'])
    result_df = df.loc[df['type'] != 2]
    result_df.loc[:, 'total_profit'] = result_df['profit'] + result_df['swap'] + result_df['commission'] + result_df['fee']
    result_df = result_df.groupby('date')['total_profit'].sum().reset_index()
    
    if profit == True:

        result_df.columns = ['date', 'growth']

    else:
        deposit  =sum(df.loc[df.type == 2].profit.tolist())
        result_df.loc[:, 'daily_balance'] =  deposit + result_df['total_profit'].cumsum()
        result_df.columns = ['date', 'daily_profit','growth']

    result_df = result_df.sort_values(by='date', ascending = False )

    return result_df

In [ ]:
def weekly_percentage_growth(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate weekly percentage growth and various related metrics.

    Parameters:
    - df (pandas.DataFrame): DataFrame containing at least the columns ['date', 'type', 'profit', 'swap', 'commission', 'fee']

    Returns:
    - pandas.DataFrame: DataFrame with weekly percentage growth and related metrics.
    """
    # create a copy of the dataframe
    df = df.copy()
    
    # Convert 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    # Calculate total deposits based on type==2 and store in variable
    deposit  = sum(df.loc[df.type == 2].profit.tolist())
    
    # Filter out rows where type is 2
    df = df.loc[df['type'] != 2]
    
    # Add a 'week' column with the week number of the year
    df['week'] = df['date'].dt.isocalendar().week
    
    # Add a 'month' column with the month of the date
    df['month'] = df['date'].dt.month
    
    # Add a 'year' column with the year of the date
    df['year'] = df['date'].dt.year
    
    # Calculate the total profit by adding the profit, swap, commission, and fee columns
    df.loc[:, 'total_profit'] = df['profit'] + df['swap'] + df['commission'] + df['fee']
    
    # Group by week, month, and year and sum the total profit for each group
    df_weekly_percentage_growth = df.groupby(by=['week', 'month', 'year'])['total_profit'].sum().reset_index()
    
    # Calculate the cumulative weekly growth
    df_weekly_percentage_growth.loc[:, 'cummulative_weekly_growth'] = df_weekly_percentage_growth['total_profit'].cumsum()
    
    # Calculate the daily balance by adding deposit to cumulative weekly growth
    df_weekly_percentage_growth.loc[:, 'daily_balance'] =  deposit + df_weekly_percentage_growth['cummulative_weekly_growth']
    
    # Calculate the percentage weekly growth based on deposit
    df_weekly_percentage_growth.loc[:, 'percentage_weekly_growth_%'] = round((df_weekly_percentage_growth['cummulative_weekly_growth'] / deposit), 3)
    
    # Calculate the cumulative weekly percentage growth based on the percentage weekly growth
    df_weekly_percentage_growth.loc[:, 'cummulative_weekly_percentage_growth_%'] = round(df_weekly_percentage_growth['percentage_weekly_growth_%'].diff() * 100, 3)
    
    # Sort the DataFrame by year, month, and week in descending order
    df_weekly_percentage_growth = df_weekly_percentage_growth.sort_values(by=['year', 'month', 'week'], ascending=False)
    
    # Slice the DataFrame to get the first two rows
    df_weekly_cum_growth = df_weekly_percentage_growth.iloc[:2]

    return df_weekly_cum_growth

df_weekly_cum_growth = weekly_percentage_growth(df)
current_growth = df_weekly_cum_growth['cummulative_weekly_percentage_growth_%'].to_list()[0]
previous_week_growth = df_weekly_cum_growth['cummulative_weekly_percentage_growth_%'].to_list()[1]
st.metric(label = 'weekly_growth', value = f"{current_growth} %", delta = f"{previous_week_growth} %" )